<a href="https://colab.research.google.com/github/lucarubini/LINKS_DeepLearning_Course/blob/main/scripts/pytorch/04_tutorialCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


torch.manual_seed(42)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [15]:
class NeuralNetwork_CNN(nn.Module):
    def __init__(self):
        super(NeuralNetwork_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# Training settings
#PyTorch MNIST Example

args={}
args['batch_size']=64 #input batch size for training
args['test_batch_size']=1000 #input batch size for testing
args['epochs']=2 #number of epochs to train
args['lr']=0.001 #learning rate
args['gamma']=0.7 #Learning rate step gamma
args['log_interval']=10000 #how many batches to wait before logging training status

In [6]:
train_kwargs = {'batch_size': args['batch_size']}
test_kwargs = {'batch_size': args['test_batch_size']}

In [7]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
dataset_train = datasets.MNIST('data', train=True, download=True,transform=transform)

dataset_test = datasets.MNIST('data', train=False, transform=transform)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset_train)
test_loader = torch.utils.data.DataLoader(dataset_test)

In [ ]:
model_cnn = NeuralNetwork_CNN().to(device)
optimizer = optim.Adam(model_cnn.parameters(), lr=args['lr'])

In [ ]:
print(model_cnn)
for param_name, var in model_cnn.named_parameters():
    print("{}: {}".format(param_name,var.shape))

In [ ]:
#Show input data
import matplotlib.pyplot as plt
plt.imshow(dataset_train.train_data[1])

In [ ]:
scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])

test(model_cnn, device, test_loader)
for epoch in range(1, args['epochs'] + 1):
  train(args, model_cnn, device, train_loader, optimizer, epoch)
  test(model_cnn, device, test_loader)
  scheduler.step()
torch.save(model_cnn.state_dict(), "mnist_cnn.pt")